In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Input

In [ ]:
X = np.load('Xtrain2_a.npy', allow_pickle=True)
y = np.load('Ytrain2_a.npy')

y = y/255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_CNN = X_train.reshape(X_train.shape[0], 7, 7, 1)
X_test_CNN = X_test.reshape(X_test.shape[0], 7, 7, 1)
# Categorize the labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

In [ ]:
model_CNN = Sequential([
    Input(shape=(7, 7, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification (crater or background)
])

model_CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_CNN.fit(X_train_CNN, y_train, epochs=10, batch_size=32, validation_data=(X_test_CNN, y_test))


In [ ]:
# prompt: now calculate the Bacc of the model

from sklearn.metrics import confusion_matrix
y_pred = model_CNN.predict(X_test_CNN)
# Assuming y_pred is the predicted labels from your model
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Extract true positives, true negatives, false positives, and false negatives
tn, fp, fn, tp = cm.ravel()

# Calculate sensitivity (recall) and specificity
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Calculate balanced accuracy
balanced_acc = (sensitivity + specificity) / 2

print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Balanced Accuracy: {balanced_acc:.4f}")